In [4]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
    
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'   # Europe
site = 'samakal_com'
c = Crawl()  # creating object

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_url = 0
unable_to_download_image = 0
publish_source = 'samakal.com'
country = 'Bangladesh'
language = 'Bangla'
images_path = []
foot_fall = c.get_foot_fall(publish_source)

months = {'জানুয়ারী': 1,
          'জানুয়ারি':1,
          'জানুয়ারি' :1,
         'ফেব্রুয়ারী': 2,
          'ফেব্রুয়ারি':2,
         'মার্চ': 3,
         'এপ্রিল': 4,
         'মে': 5,
         'জুন': 6,
         'জুলাই': 7,
         'অগাস্ট': 8,
         'সেপ্টেম্বর': 9,
         'অক্টোবর': 10,
         'নভেম্বর': 11,
         'ডিসেম্বর': 12}

numbers_dic = {  
             '০': '0',
             '১': '1',
             '২': '2',
             '৩': '3',
             '৪': '4',
             '৫': '5',
             '৬': '6',
             '৭': '7',
             '৮': '8',
             '৯': '9'
           }

am_pm_dic = {
    'পিএম': 'pm',
    'এএম' : 'am'
}

def translate_number(number, numbers_dic):
    return ''.join(map(lambda x:numbers_dic[x], number))

# archive page
url_pages = c.download_page('http://samakal.com/list/all')
print(url_pages)
#class="fa fa-clock-o">
for i in url_pages.split('class="fa fa-clock-o">')[1:]:
    # source_link
    source_link = c.scrap('href="(.*?)"', i)
    source_link = c.scrap('(.*)/', source_link)
    print(source_link)
    continue
    # handle duplicates
    source_link_query = {'source_link':source_link}
    dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
    if dic:
        duplicate_data += 1
        continue
    
    time.sleep(random.randint(1,3))
     
    page = c.download_page(source_link)
    if page.startswith('Unable to fetch'):
        logger.info(page) # writes error message with error code
        unable_to_fetch_url += 1
        continue    

    # source_headline
    source_headline = c.scrap("<title>(.*?)<", page)
    
    # skip if headline not found
    if not source_headline:
        logger.info(f'Skipping due to headline {source_link}\n')
        # print(f'Skipping due to headline {source_link}\n')
        skipped_due_to_headline += 1
        continue
        
    # Date and time
    pub_date, publish_time = '', ''

    try:
        date_time_str = c.scrap('class="fa fa-calendar">.*?,(.*?)<', page)

        date_time_str = date_time_str.split()
        day = translate_number(date_time_str[0] , numbers_dic)
        month = months[date_time_str[1]]
        year =  date_time_str[2]
        year = translate_number(year, numbers_dic)
        month = str(month)
        if len(month)==1:
            month = '0' + str(month)

        date_time_str = f'{day}{month}{year}000000'
        date_time_obj = datetime.strptime(date_time_str, '%d%m%Y%H%M%S')
        ist_date_time = date_time_obj - timedelta(hours = 0,minutes = 0)  # Bangladesh is 30 minutes ahead of India
        ist_date_time = ist_date_time.strftime('%Y-%m-%d-%H-%M-%S')
        pub_date = ist_date_time[:10]
        publish_time = ist_date_time[11:]  
    except:
        pass

    # skip null date
    if not pub_date:
        logger.info(f'Skipping due to date {source_link}\n')
        skipped_due_to_date += 1
        continue
    
    # break if date is not today's date
    if pub_date != created_on:
        break    

    # source_content          
    source_content = c.scrap('class="contentBody">(.*?)<div\s*class="row hidden-print\s*">', page)
 
    
    source_content = c.strip_html(source_content)

    # skip if content not found
    if not source_content:
        logger.info(f'Skipping due to content {source_link}\n')
        # print(f'Skipping due to content {source_link}\n')
        skipped_due_to_content += 1
        continue
    
    # journalist
    journalist = c.scrap('class="fa fa-user">(.*?)<div\s*style="padding-top', page)
    journalist = c.strip_html(journalist)
    
    if not journalist: journalist = 'NA'

    logger.info(f'Fetching {source_link}\n')
    # print(f'Fetching {source_link}\n')
   
    # current date and time 
    harvest_time = datetime.now().strftime("%H:%M:%S")

    # temp link
    temp_link = source_link

    # headline and content 
    headline = source_headline
    content = source_content

    # overall_tonality
    overall_tonality = ''

    # word count
    word_count = len((source_headline + ' ' + source_content).split())

    html_content = ''

    # image_urls
    image_urls = []
        
#     print('source_link', source_link, '\n')
#     print('source_headline', source_headline,'\n')
#     print('pub_date', pub_date,'\n')
#     print('publish_time', publish_time,'\n')
#     print('journalist', journalist,'\n')
#     print('source_content', source_content,'\n')
#     print('html_content', html_content, '\n')
#     print('image_urls', image_urls, '\n')
    
    # storing the above data in a dictionary
    clientdata ={
                    "client_master" : client_id, 
                    "articleid":client_id,
                    "medium":'Web' ,
                    "searchkeyword":[],
                    "entityname" : [] ,
                    "process_flage":"1",
                    "na_flage":"0",
                    "na_reason":"",
                    "qc_by":"",
                    "qc_on":"",
                    "location":"",
                    "spokeperson":"",
                    "quota":"",
                    "overall_topics":"",
                    "person":"",
                    "overall_entites":"",
                    "overall_tonality": overall_tonality,
                    "overall_wordcount":word_count,
                    "article_subjectivity":"",
                    "article_summary":"",
                    "pub_date":pub_date,
                    "publish_time":publish_time,
                    "harvest_time":harvest_time,
                    "temp_link":temp_link,
                    "publish_source": publish_source,
                    "programme":'null',
                    "feed_class":"News",
                    "publishing_platform":"",
                    "klout_score":"",
                    "journalist":journalist,
                    "headline":headline,
                    "content":content,
                    "source_headline":source_headline,
                    "source_content":source_content,
                    "language":language,
                    "presence":'null',
                    "clip_type":'null',
                    "prog_slot":'null',
                    "op_ed":'0',
                    "location_mention":'',
                    "source_link":source_link,
                    "author_contact":'',
                    "author_emailid":'',
                    "author_url":'',
                    "city":'',
                    "state":'',
                    "country":country,
                    "source":publish_source,
                    "foot_fall":foot_fall,
                    "created_on":created_on,
                    "active":'1',
                    'crawl_flag':2,
                    "images_path":images_path,
                    "html_content":html_content
                } 

#     cl_data.insert_one(clientdata)  # get object id and insert data
    no_of_data += 1


logger.info('Iteration complete\n')   

logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch url: {unable_to_fetch_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'country: {country}\n')
logger.info(f'language: {language}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')

Unable to fetch http://samakal.com/list/all Error code: 503

